# Download AMSR data

DataAccess in general
- https://podaac.jpl.nasa.gov/dataset/AMSR2-REMSS-L3U-v8a

OpenDAP-Link
- https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L3U/AMSR2/REMSS/v8a/

In [1]:
import xarray as xr
import sys
sys.path.append('../../BCO/')
from general import *

In [16]:
link = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L3U/AMSR2/REMSS/v8a/2017/005/20170105000000-REMSS-L3U_GHRSST-SSTsubskin-AMSR2-f34_20170105v8-v02.0-fv01.0.nc'

In [30]:
ds = xr.open_dataset(link, drop_variables=['dt_analysis','sses_bias','sses_standard_deviation',
                                           'diurnal_amplitude','cool_skin','wind_speed',
                                           'sea_ice_fraction','proximity_confidence', 'rain_rate', 'sst_dtime'])

In [31]:
lat = [5,60]
lon = [-75,-5]

In [32]:
ds_sel = ds.sel({'lat':slice(lat[0],lat[1]), 'lon':slice(lon[0],lon[1])})

In [33]:
ds_sel = compress_all_variables(ds_sel)

In [34]:
ds_sel.to_netcdf('test.nc')

In [29]:
ds_sel

<xarray.Dataset>
Dimensions:                  (lat: 220, lon: 260, time: 2)
Coordinates:
  * time                     (time) datetime64[ns] 2017-01-05 2017-01-06
  * lat                      (lat) float32 5.125 5.375 5.625 ... 59.625 59.875
  * lon                      (lon) float32 -69.875 -69.625 ... -5.375 -5.125
Data variables:
    sea_surface_temperature  (time, lat, lon) float32 ...
    sst_dtime                (time, lat, lon) float64 ...
    dt_analysis              (time, lat, lon) float32 ...
    water_vapor              (time, lat, lon) float32 ...
    cloud_liquid_water       (time, lat, lon) float32 ...
    quality_level            (time, lat, lon) float32 ...
    rejection_flag           (time, lat, lon) int16 ...
    confidence_flag          (time, lat, lon) int16 ...
Attributes:
    Conventions:                CF-1.6, Unidata Observation Dataset v1.0
    title:                      Sea Surface Temperature from AMSR2 onboard GC...
    summary:                    Sea Surf

## Batch-Download

In [25]:
import xarray as xr
import sys
import datetime as dt
import pandas as pd
import tqdm
sys.path.append('../../BCO/')
from general import *

In [22]:
PATH_BASE = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L3U/AMSR2/REMSS/v8a/{year}/{jday:03g}/'
FILE_FMT = '{yyyymmdd}000000-REMSS-L3U_GHRSST-SSTsubskin-AMSR2-f34_{yyyymmdd}v8-v02.0-fv01.0.nc'

OUTPUT_PATH = './'
OUTPUT_FMT = 'AMSR__L3__{yyyymmdd}.nc'

lat = [5,60]
lon = [-75,-5]

In [23]:
dates = pd.date_range(dt.datetime(2013,1,1), dt.datetime(2014,1,1))

In [29]:
def add_metadata(ds):
    """
    Add additional metadata to netcdf files
    """
    ds.attrs['created_with'] = 'download_AMSRsubsets.ipynb'
    ds.attrs['created_on'] = dt.datetime.now().strftime('%Y%m%d %H:%M')
    
    return ds

In [ ]:
for date in tqdm.tqdm(dates):
    jday = date.dayofyear
    year = date.year
    yyyymmdd = date.strftime('%Y%m%d')
    
    path = PATH_BASE.format(year=year, jday=jday)
    link = path+FILE_FMT.format(yyyymmdd=yyyymmdd)
    try:
        ds = xr.open_dataset(link, drop_variables=['dt_analysis','sses_bias','sses_standard_deviation',
                                               'diurnal_amplitude','cool_skin','wind_speed',
                                               'sea_ice_fraction','proximity_confidence', 'rain_rate', 'sst_dtime'])
    except KeyError:
        print(f'File {link} not found.')
        continue
    else:
        ds_sel = ds.sel({'lat':slice(lat[0],lat[1]), 'lon':slice(lon[0],lon[1])})
        ds_sel = compress_all_variables(ds_sel)
        ds_sel = add_metadata(ds_sel)
        ds_sel.to_netcdf(OUTPUT_PATH+OUTPUT_FMT.format(yyyymmdd=yyyymmdd))

In [34]:
xr.open_mfdataset('AMSR__L3__*.nc', concat_dim='time', combine='by_coords')

<xarray.Dataset>
Dimensions:                  (lat: 220, lon: 280, time: 258)
Coordinates:
  * lat                      (lat) float32 5.125 5.375 5.625 ... 59.625 59.875
  * lon                      (lon) float32 -74.875 -74.625 ... -5.375 -5.125
  * time                     (time) datetime64[ns] 2013-01-01 ... 2013-05-10T20:48:00
Data variables:
    sea_surface_temperature  (time, lat, lon) float32 dask.array<shape=(258, 220, 280), chunksize=(2, 220, 280)>
    water_vapor              (time, lat, lon) float32 dask.array<shape=(258, 220, 280), chunksize=(2, 220, 280)>
    cloud_liquid_water       (time, lat, lon) float32 dask.array<shape=(258, 220, 280), chunksize=(2, 220, 280)>
    l2p_flags                (time, lat, lon) int16 dask.array<shape=(258, 220, 280), chunksize=(2, 220, 280)>
    quality_level            (time, lat, lon) float32 dask.array<shape=(258, 220, 280), chunksize=(2, 220, 280)>
    rejection_flag           (time, lat, lon) int16 dask.array<shape=(258, 220, 280), ch